In [4]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import time
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import torch
import tensorflow as tf
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
path = '/content/drive/MyDrive/Group 1/Stemmed.pkl'

with open(path, 'rb') as file:
    data_stemmed = pickle.load(file)


path = '/content/drive/MyDrive/Group 1/DoctoVecModel.pkl'

with open(path, 'rb') as file:
    data_doctovec = pickle.load(file)


path = '/content/drive/MyDrive/Group 1/DistilEmbed.pkl'

with open(path, 'rb') as file:
    data_distilembed = pickle.load(file)


path = '/content/drive/MyDrive/Group 1/data_clean.pkl'

with open(path, 'rb') as file:
    data_clean = pickle.load(file)

# print 1st row of dataset
# print(data_stemmed[0])
# print(data_doctovec[0])
# print(data_distilembed[0])
# print(data_clean[0])

car wonder anyon could enlighten car saw dai door sport car look late earli call bricklin door realli small addit front bumper separ rest bodi know anyon tellm model name engin spec year product car made histori whatev info funki look car pleas mail thank
0        0.041550
1        0.012731
2        0.295469
3        0.135433
4        0.052115
           ...   
11309    0.208519
11310   -0.039752
11311    0.109750
11312   -0.154094
11313   -0.005902
Name: 0, Length: 11314, dtype: float32
0        0.167505
1       -0.112607
2       -0.168808
3       -0.118984
4       -0.108023
           ...   
11309   -0.208917
11310   -0.003179
11311   -0.063529
11312   -0.219597
11313    0.130606
Name: 0, Length: 11314, dtype: float32
car wondering anyone could enlighten car saw day door sports car looked late early called bricklin doors really small addition front bumper separate rest body know anyone tellme model name engine specs years production car made history whatever info funky looking car pl

In [23]:
df_stemmed = pd.DataFrame(data_stemmed)
df_stemmed.rename(columns={0: 'preprocessed'}, inplace=True)
#df_stemmed

df_clean = pd.DataFrame(data_clean)
df_clean.rename(columns={0: 'preprocessed'}, inplace=True)
#df_clean


,preprocessed
0,car wondering anyone could enlighten car saw d...
1,clock poll final call final call clock reports...
2,questions well folks mac plus finally gave gho...
3,weitek robert kyanko rob rjck uucp wrote abrax...
4,shuttle launch question article cowcb world st...
...,...
11309,migraines scans nyeda cnsvax uwec edu david ny...
11310,screen death mac plus old mac mac plus problem...
11311,mounting cpu cooler vertical case installed cp...
11312,sphere points article qkgbuinnsn shelley washi...


In [25]:
df_newsgroup = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df_newsgroup

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [38]:
df_newsgroup_2 = df_newsgroup.drop(columns=['content'])
df_clean_concat = pd.concat([df_clean, df_newsgroup_2], axis = 1)
df_stemmed_concat = pd.concat([df_stemmed, df_newsgroup_2], axis = 1)

#df_clean_concat

In [49]:
df_clean_concat_filtered = df_clean_concat[df_clean_concat['target_names'].isin(['soc.religion.christian', 'rec.sport.hockey', 'talk.politics.mideast', 'rec.motorcycles'])]
df_clean_concat_filtered['target2']=df_clean_concat['target'].apply(lambda x: 0 if x==8 else (1 if x==10 else (2 if x==15 else 3)))
df_stemmed_concat_filtered = df_stemmed_concat[df_stemmed_concat['target_names'].isin(['soc.religion.christian', 'rec.sport.hockey', 'talk.politics.mideast', 'rec.motorcycles'])]

df_stemmed_concat_filtered


<ipython-input-49-280bb9d9a642>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_concat_filtered['target2']=df_clean_concat['target'].apply(lambda x: 0 if x==8 else (1 if x==10 else (2 if x==15 else 3)))


,preprocessed,target,target_names
10,recommend duc worth ducati gt much line ducati...,8,rec.motorcycles
21,nhl team captain articl apr samba oit unc edu ...,10,rec.sport.hockey
28,pantheism environment articl apr atho rutger e...,15,soc.religion.christian
33,isra expans lust articl spam math adelaid edu ...,17,talk.politics.mideast
35,goali mask articl netnew upenn edu kkeller mai...,10,rec.sport.hockey
...,...,...,...
11298,nhl team milwauke read report possibl nhl move...,10,rec.sport.hockey
11299,turkei cypru bosnia serbia greec armenia azeri...,17,talk.politics.mideast
11300,arrog christian previou articl phsd vaxc monas...,15,soc.religion.christian
11305,hezbollah apr yuma acn colost edu repli long l...,17,talk.politics.mideast


Target2 is there to classify the topics into 4 categories

In [44]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Binarizer

In [62]:
# Relative frequency
tvec = TfidfVectorizer(max_df=0.7, min_df=0.1,use_idf=False, norm='l1')
data_stem2 = tvec.fit_transform(df_stemmed_concat_filtered['preprocessed'])

# convert matrix into df
df_stem2 = pd.DataFrame(data_stem2)

# concatenate dfs
# Reset the index
df_stem2 = df_stem2.reset_index(drop=True)
df_stemmed_concat_filtered = df_stemmed_concat_filtered.reset_index(drop=True)

# Concatenate the dataframes
df_stemmed_tvec = pd.concat([df_stem2, df_stemmed_concat_filtered], axis = 1)
df_stemmed_tvec

,0,preprocessed,target,target_names
0,"(0, 70)\t0.07142857142857142\n (0, 130)\t0....",recommend duc worth ducati gt much line ducati...,8,rec.motorcycles
1,"(0, 118)\t0.09523809523809523\n (0, 114)\t0...",nhl team captain articl apr samba oit unc edu ...,10,rec.sport.hockey
2,"(0, 118)\t0.015625\n (0, 86)\t0.015625\n (...",pantheism environment articl apr atho rutger e...,15,soc.religion.christian
3,"(0, 130)\t0.03571428571428571\n (0, 129)\t0...",isra expans lust articl spam math adelaid edu ...,17,talk.politics.mideast
4,"(0, 118)\t0.058823529411764705\n (0, 58)\t0...",goali mask articl netnew upenn edu kkeller mai...,10,rec.sport.hockey
...,...,...,...,...
2356,"(0, 130)\t0.038461538461538464\n (0, 58)\t0...",nhl team milwauke read report possibl nhl move...,10,rec.sport.hockey
2357,"(0, 130)\t0.017857142857142856\n (0, 118)\t...",turkei cypru bosnia serbia greec armenia azeri...,17,talk.politics.mideast
2358,"(0, 129)\t0.007751937984496124\n (0, 58)\t0...",arrog christian previou articl phsd vaxc monas...,15,soc.religion.christian
2359,"(0, 70)\t0.011627906976744186\n (0, 130)\t0...",hezbollah apr yuma acn colost edu repli long l...,17,talk.politics.mideast
